In [18]:
import os
import pandas as pd

year_dir = "./FTGA/data/FinTech Germany Award 2020"
year_num = 2020

# Read the data files
general_df = pd.read_excel(os.path.join(year_dir, "general.xlsx"))

if os.path.exists(os.path.join(year_dir, "scores.xlsx")):
    scores_df = pd.read_excel(os.path.join(year_dir, "scores.xlsx"))
    scores_df = scores_df.dropna(subset=["Startup ID"])
else:
    scores_df = pd.DataFrame(columns=["Startup ID", "Average evaluation"])

# Merge the data files
merged_df = general_df.merge(scores_df, on="Startup ID", how="left")

# Split scored data
scored_df = merged_df.dropna(subset=["Average evaluation"])
scored_df["score_category"] = scored_df["Average evaluation"].apply(lambda x: ("HIGH" if x >= 5 else "MID") if x > 3 else "LOW")

high_scores = scored_df[scored_df["Average evaluation"] > 4]
low_scores = scored_df[scored_df["Average evaluation"] <= 4]

train_high = high_scores.sample(frac=0.8, random_state=38)
test_high = high_scores.drop(train_high.index)
train_low = low_scores.sample(frac=0.8, random_state=38)
test_low = low_scores.drop(train_low.index)

train_scored = pd.concat([train_high, train_low])
test_scored = pd.concat([test_high, test_low])

# Identify entries in general_df that do not have a score
valid_df = general_df[~general_df["Startup ID"].isin(scored_df["Startup ID"])]

# Save split data to CSV files
train_file = os.path.join(year_dir, f"train.csv")
test_file = os.path.join(year_dir, f"test.csv")
valid_file = os.path.join(year_dir, f"valid.csv")

train_scored[["Startup ID"]].to_csv(train_file, index=False)
test_scored[["Startup ID"]].to_csv(test_file, index=False)
valid_df[["Startup ID"]].to_csv(valid_file, index=False)
